# IMPORT

In [1]:
import requests
from bs4 import BeautifulSoup
import os

# FUNCTIONS

In [2]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            elif 'selezionare' in response2.text.lower():
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','ente mancante / possibile altro errore']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text: 
                    compila = 'si'

                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

In [3]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    #print(row_number)
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result=''
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        rep = str(numero) + ' - ' + str(anno)
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': ufficio,
                            'natura': natura,
                            'ente': ente,
                            'repertorio': rep,
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number} , ente : {ente} , natura : {natura} , debitore : {debitore}')
        print(f"Error occurred: {e}")
    return results

# tipo_scraper('1','de','ppt','TQE',['001'],20,2,1)

# PULIZIA INPUT

In [4]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
import re
today = date.today()


In [5]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

# Cambiare file di input!


In [6]:
input0 = pd.read_excel('Input/Raw/Perimetro Giugno 24.xlsx')

In [7]:
input = input0.copy()

In [8]:
len(input)

3136

In [9]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [10]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [11]:
input = input.drop_duplicates()
input = clean(input)

Split did not produce the expected number of columns for some rows.


In [12]:
len(input) # non deve diminuire

3136

In [13]:
mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'",

    "gaeta":'formia',
    "marano di napoli":'napoli',
    "fondi":'formia',
    "mascalucia":'catania',
    "pomigliano d'arco":'casoria',
    "acri":'cosenza',
    "cecina":'livorno',
    "tricase":'casarano',
    "castiglione del lago":'perugia',
    "rodi garganico":'san severo',
    "pontremoli":'aulla',
    "squillace":'catanzaro',
    "carinola":"sessa aurunca - teano",
    "bisceglie":'trani',
    "biancavilla":'catania',
    "pescina":'avezzano',
    "lauro":'avellino',
    "tortoli'":'lanusei',
    "piazza armerina":'enna',
    "petilia policastro": 'crotone',
    "segni":'velletri',
    "corato":'bari',
    "ciro'":'crotone',
    "corigliano calabro":'rossano',
    "afragola":'casoria',
    "terracina":'latina',
    "monopoli":'bari',
    "sant'angelo di brolo":'messina',
    "airola":'benevento',
    "nulvi":'sassari',
    "ramacca":'caltagirone',
    "trinitapoli":'barletta'
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input
input = input.drop_duplicates()

In [14]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [15]:
input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

input = input.drop_duplicates()

Se questo non è [], sistemare il mapping

In [16]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

In [17]:
input['Debitore'] = (input['Debitore']
                     .str.replace('-', ' ')
                     .str.replace(r'\s+', ' ', regex=True)
                     .str.strip()
                    )


In [18]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [19]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)


In [20]:
len(input)

4008

In [21]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente 1','Cliente 2','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente 1','Cliente 2','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [22]:
input = input.drop(columns={'category'})
input['code_ente'] = input['code_ente'].apply(tuple)
input = input.drop_duplicates()
input['code_ente'] = input['code_ente'].apply(list)

In [23]:
input = input[(input['year']!='')]
len(input)

7870

se ci sono risultati manda a daniele

In [24]:
x = input[(input['year']!='')]
x['year'] = x['year'].astype(int)
x[(x['year']>2024) | (x['year']<1900)]



,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente 1,Cliente 2,tipo_ricerca,code_office,office,code_ente,nr,year
50,AA1700000339_04022021,AOUACHE HASSAN,giudice,mantova,A2A,A2A S.p.a.,monitoria,TNH,mantova ut dpmn,"[004, 006, 007]",1631,2921
1728,BZ300858743_1005_20230324,SCORDINO FRANCESCO,tribunale,catania,BNL,BNL-Zeus,monitoria,TXN,catania ut dpct,"[001, 002, 006, 007, 008, 009, 012, 014, 017]",4237,203
1741,BZ303252626_1005_20230609,FERRARA RITA,tribunale,napoli,BNL,BNL-Zeus,monitoria,TER,napoli 1 ut dpna1,"[001, 003, 004, 018, 019, 020, 021, 022, 023, ...",35638,13207
1744,BZ303252626_1005_20230609,FERRARA RITA,tribunale,napoli,BNL,BNL-Zeus,monitoria,TES,napoli 2 ut dpna1,[001],35638,13207
1747,BZ303252626_1005_20230609,FERRARA RITA,tribunale,napoli,BNL,BNL-Zeus,monitoria,TET,napoli 3 ut dpna2,"[001, 002, 003, 004, 005, 006, 007, 016, 017, ...",35638,13207
3719,EN812414720_03062014,DE MICHELE PIETRO,giudice,barcellona pozzo di gotto,ENI,ENI S.p.A.,monitoria,TXH,barcellona pozzo di gotto ut dpme,"[002, 003, 007, 008, 011, 012]",8,2918
4777,EN832701932_13102023,EBRAHIM ASHRAF,giudice,milano,Eni,ENI S.p.A.,monitoria,TNL,milano 1 ut dpmi1,"[006, 016]",25343,2029
4778,EN832701932_13102023,EBRAHIM ASHRAF,giudice,milano,Eni,ENI S.p.A.,monitoria,TNL,milano 1 ut dpmi1,"[006, 016]",30316,2029
10537,UBS380208,LICITRA GIULIA,giudice,ancona,Sirio,SIRIO NPL S.R.L.,monitoria,TQD,ancona ut dpan,"[005, 006]",1177,223


In [25]:
#4498   ??? 
input = input[(input['year']!='')]
input['year'] = input['year'].astype(int)
input = input[(input['year']<=2024) & (input['year']>1900)]
len(input)

7861

In [26]:
input.to_excel(f'Input/Cleaned/{today}_input_v1.xlsx',index=False)

In [27]:
"""input_vecchio = pd.read_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',dtype={'nr':str,'year':str})

input_vecchio['Debitore'] = (input_vecchio['Debitore']
                     .str.replace('-', ' ')
                     .str.replace(r'\s+', ' ', regex=True)
                     .str.strip()
                    )"""


"input_vecchio = pd.read_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',dtype={'nr':str,'year':str})\n\ninput_vecchio['Debitore'] = (input_vecchio['Debitore']\n                     .str.replace('-', ' ')\n                     .str.replace(r'\\s+', ' ', regex=True)\n                     .str.strip()\n                    )"

In [28]:
"""input['code_ente'] = input['code_ente'].astype(str)
input_vecchio['code_ente'] = input_vecchio['code_ente'].astype(str)"""

"input['code_ente'] = input['code_ente'].astype(str)\ninput_vecchio['code_ente'] = input_vecchio['code_ente'].astype(str)"

In [29]:
"""diff_df = input.merge(input_vecchio, how='outer', indicator=True)
# Filter the rows that are different
diff_rows = diff_df[diff_df['_merge'] == 'left_only']
diff_rows['code_ente'] = diff_rows['code_ente'].apply(eval)
diff_rows"""

"diff_df = input.merge(input_vecchio, how='outer', indicator=True)\n# Filter the rows that are different\ndiff_rows = diff_df[diff_df['_merge'] == 'left_only']\ndiff_rows['code_ente'] = diff_rows['code_ente'].apply(eval)\ndiff_rows"

# SCRAPER

Modificare new_df, 500 per volta

In [39]:
a=100
b=200
# b = "fine"

# new_df = input[a:]

new_df = input[a:b]
results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], 
                                    anno=row['year'],
                                    numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

# PULIZIA RISULTATI

In [40]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()

In [41]:
cliente = input0[['NR_Contratto','Cliente 1','Cliente 2']]
df = pd.merge(df,cliente,how='left', left_on='contratto',right_on='NR_Contratto' )
df.drop(columns=['NR_Contratto'], inplace=True)
df['Cliente 1'] = df['Cliente 1'].str.upper()
df['Cliente 2'] = df['Cliente 2'].str.upper()
df = df.drop_duplicates()

# Check visivi

In [42]:
# Righe con risultati
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente 1,Cliente 2
61,AA1700001034_04022021,ABF SRL UNIPERSONALE,REGGIO EMILIA UT DPRE,DECRETO INGIUNTIVO n. 2136 del 2021,TRIB. R.E. UFF. GIUDICE UNICO,497/22 - 2022,LOCA TOP,.,ATTO REGISTRATO,no,A2A,A2A S.P.A.
412,AA1700001404_04022021,SINGH SUKHVINDER,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 451 del 2022,GIUDICE DI PACE-MANTOVA,665 - 2022,APRICA S.P.A.,SINGH,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
448,AA1700001500_04022021,DAHBI ABDELLAH,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 235 del 2022,TRIBUNALE DI MANTOVA,301 - 2022,ZANCHI,SARZI AMADE',STAMPATO AVVISO DI LIQUIDAZIONE,no,A2A,A2A S.P.A.
450,AA1700001500_04022021,DAHBI ABDELLAH,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 109 del 2022,TRIBUNALE DI MANTOVA,959/2023 - 2022,TEA ENERGIA S.R.L.,ALESSANDRI,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
452,AA1700001507_04022021,FAGGIONI ERICA,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 515 del 2022,GIUDICE DI PACE-MANTOVA,663 - 2022,APRICA S.P.A.,FAGGIONI,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
476,AA1700001525_04022021,OMRI MOHAMED,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 526 del 2022,GIUDICE DI PACE-MANTOVA,662 - 2022,APRICA S.P.A.,OMRI,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
524,AA1700001564_04022021,ALI IMRAN,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 868 del 2022,TRIBUNALE DI MANTOVA,97 - 2022,IFIS NPL INVESTING S.P.A.,POZZI,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
526,AA1700001564_04022021,ALI IMRAN,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 536 del 2022,TRIBUNALE DI MANTOVA,1050 - 2022,M.T. MANTOVANA TERMOSANITARIA,SALVATERRA,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
534,AA1700001596_04022021,SINGH SUKHWINDER,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 664 del 2022,GIUDICE DI PACE-MANTOVA,124 - 2023,TEA ENERGIA S.R.L.,NDOYE,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.
558,AA1700001605_04022021,SINGH GURDEV,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 676 del 2022,GIUDICE DI PACE-MANTOVA,313 - 2022,SERVIZIO IDRICO INTEGRATO COMU,ANDREANO,ATTO IN FASE DI TASSAZIONE,no,A2A,A2A S.P.A.


In [43]:
# Guardare i nomi in format 'name +1'
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente 1,Cliente 2


# SALVATAGGIO OUTPUT

In [44]:
df['attore'] = (df['attore']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['convenuto'] = (df['convenuto']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['debitore'] = (df['debitore']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['Cliente 1'] = (df['Cliente 1']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
df['Cliente 2'] = (df['Cliente 2']
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )

In [45]:
df['convenuto'] = df['convenuto'] .str.upper()
df['debitore'] = df['debitore'] .str.upper()
df['attore'] = df['attore'] .str.upper()
df['Cliente 1'] = df['Cliente 1'] .str.upper()
df['Cliente 2'] = df['Cliente 2'] .str.upper()

In [46]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()

In [47]:
# df = pd.DataFrame(results.explode().tolist())
target_dir= f'Output/{str(today)}'
os.makedirs(target_dir, exist_ok=True)
df.drop(columns={'compila'}).to_excel(f'Output/{str(today)}/{str(today)}_output_{a}_{b}_v1.xlsx',index=False)


# Da modificare: Capire come cambiare Cliente in cliente 1 / 2 !!

In [42]:
df = df[df['stato']!='-']
mask = df.apply(lambda row: (row['Cliente 1'] in row['attore']) 
                   or (row['attore'] in row['Cliente 1']) or
                    (row['Cliente 2'] in row['attore']) 
                   or (row['attore'] in row['Cliente 2']) or
                     (row['convenuto'] in row['debitore'])
                   or (row['debitore'] in row['convenuto']), axis=1)

df['match']='no'
df.loc[mask, 'match'] = 'si'

df['esito'] = '?'
df.loc[(df['match'] == 'si') & (df['compila'] == 'si'), 'esito'] = 'f24'
df.loc[(df['match'] == 'si') & (df['compila'] == 'no'), 'esito'] = 'esito ok'
df.loc[(df['match'] == 'no') & (df['compila'] == 'no'), 'esito'] = 'verifica dati'
df.loc[(df['match'] == 'no') & (df['compila'] == 'si'), 'esito'] = 'verifica dati / possibile f24'


df.to_excel(f'Output/{str(today)}/{str(today)}_esiti_positivi_{a}_{b}_v2.xlsx',index=False)

C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_21672\2563999687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['match']='no'
C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_21672\2563999687.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['esito'] = '?'


Uniamo tutti i file di output

In [4]:
df = pd.DataFrame()
folder_path = f'Output/{str(today)}'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'output' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel(f'Output/{str(today)}/{str(today)}_output_totale.xlsx',index= False)
            

In [5]:
df = pd.DataFrame()
folder_path = f'Output/{str(today)}'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'esiti' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel(f'Output/{str(today)}/{str(today)}_esiti_positivi.xlsx',index=False)